In [1]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")

/mbz/users/shibo.hao/miniforge3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B").cuda()

In [3]:
print(tokenizer.chat_template)

{% if not add_generation_prompt is defined %}{% set add_generation_prompt = false %}{% endif %}{% set ns = namespace(is_first=false, is_tool=false, is_output_first=true, system_prompt='') %}{%- for message in messages %}{%- if message['role'] == 'system' %}{% set ns.system_prompt = message['content'] %}{%- endif %}{%- endfor %}{{bos_token}}{{ns.system_prompt}}{%- for message in messages %}{%- if message['role'] == 'user' %}{%- set ns.is_tool = false -%}{{'<｜User｜>' + message['content']}}{%- endif %}{%- if message['role'] == 'assistant' and message['content'] is none %}{%- set ns.is_tool = false -%}{%- for tool in message['tool_calls']%}{%- if not ns.is_first %}{{'<｜Assistant｜><｜tool▁calls▁begin｜><｜tool▁call▁begin｜>' + tool['type'] + '<｜tool▁sep｜>' + tool['function']['name'] + '\n' + '```json' + '\n' + tool['function']['arguments'] + '\n' + '```' + '<｜tool▁call▁end｜>'}}{%- set ns.is_first = true -%}{%- else %}{{'\n' + '<｜tool▁call▁begin｜>' + tool['type'] + '<｜tool▁sep｜>' + tool['functio

In [12]:
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-1.5B")

In [13]:
inputs = tokenizer.apply_chat_template(
    [
        {
            "role": "user",
            "content": "What is the capital of the moon?"
        }
    ],
    add_generation_prompt=True,
)

In [14]:
inputs

[151644,
 8948,
 198,
 2610,
 525,
 264,
 10950,
 17847,
 13,
 151645,
 198,
 151644,
 872,
 198,
 3838,
 374,
 279,
 6722,
 315,
 279,
 17788,
 30,
 151645,
 198,
 151644,
 77091,
 198]

In [15]:
tokenizer.decode(inputs)

'<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\nWhat is the capital of the moon?<|im_end|>\n<|im_start|>assistant\n'

In [8]:
import torch

In [9]:
outputs = model.generate(torch.tensor(inputs).unsqueeze(0).cuda(), max_new_tokens=1000)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [11]:
print(tokenizer.decode(outputs[0, :]))

<｜begin▁of▁sentence｜><｜User｜>What is the capital of the moon?<｜Assistant｜><think>
Okay, so I'm trying to figure out the capital of the moon. Hmm, that's an interesting question. I know that on Earth, we have cities like New York and Tokyo, which are the capitals. But the moon is a celestial body, not a planet, so it doesn't have a capital in the traditional sense. Maybe the question is a play on words or something else.

Let me think. The word "moon" has "o" as the second letter. If I capitalize that, it would become " moons," but that doesn't make much sense. Maybe it's a trick question. Perhaps the answer is that there isn't a capital, or that the moon doesn't have one. Or maybe it's referencing something else, like a fictional city or a game.

I should consider different interpretations. One possibility is that the question is asking about the capital of the Moon, which is a hypothetical planet. If it were a planet, it would be a city, but since it's a moon, perhaps the answer is th

In [31]:
print(tokenizer.decode(outputs[0, : 20]))

<｜begin▁of▁sentence｜><｜User｜>What is the capital of the moon?<｜Assistant｜><think>
Okay, so I need to figure


In [32]:
# print a token-idx map of the first 20 tokens

for i, token in enumerate(outputs[0, : 20]):
    print(f"{i}: {tokenizer.decode(token)}")



0: <｜begin▁of▁sentence｜>
1: <｜User｜>
2: What
3:  is
4:  the
5:  capital
6:  of
7:  the
8:  moon
9: ?
10: <｜Assistant｜>
11: <think>
12: 

13: Okay
14: ,
15:  so
16:  I
17:  need
18:  to
19:  figure
